In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler

desktop    = Path.home() / "Desktop"
excel_path = desktop / "data2.xlsx"
df = pd.read_excel(excel_path, sheet_name="Sheet1", header=None)


u_drive   = df.iloc[2:, 0].astype(float).to_numpy()  
angle_rad = df.iloc[2:, 1].astype(float).to_numpy() 
T = len(u_drive)

angle_rad = np.mod(angle_rad, 2*np.pi)

dt   = 0.01                   
time = np.arange(T) * dt

sin_ang = np.sin(angle_rad)
cos_ang = np.cos(angle_rad)


U = np.stack([sin_ang, cos_ang, u_drive], axis=1)   # (T,3)
Y = angle_rad.reshape(-1, 1)         # (T,1)



idx_split = min(int(800.0/dt), T)
U_train, U_test = U[:idx_split], U[idx_split:]
Y_train, Y_test = Y[:idx_split], Y[idx_split:]
time_test       = time[idx_split:]


scaler_U  = StandardScaler().fit(U_train)
scaler_Y  = StandardScaler().fit(Y_train)
U_train_s = scaler_U.transform(U_train)
Y_train_s = scaler_Y.transform(Y_train)
U_test_s  = scaler_U.transform(U_test)

class ESN:
    def __init__(self, in_dim, res_dim, out_dim,
                 spectral_radius=1.00, leaking_rate=0.4,
                 sparsity=0.1, ridge_param=1e-8, random_state=None):
        rs = np.random.RandomState(random_state)
        self.alpha = leaking_rate
        self.Win = rs.uniform(-1,1,(res_dim, in_dim))
        W = rs.uniform(-1,1,(res_dim, res_dim))
        W[rs.rand(*W.shape) > sparsity] = 0
        W *= spectral_radius / np.max(np.abs(np.linalg.eigvals(W)))
        self.W = W
        self.ridge = ridge_param
        self.res_dim = res_dim

    def _update(self, x, u):
        x_new = np.tanh(self.Win.dot(u) + self.W.dot(x))
        return (1 - self.alpha)*x + self.alpha * x_new

    def fit(self, U, Y, washout=40):
        T = U.shape[0]
        X = np.zeros((T - washout, self.res_dim))
        x = np.zeros(self.res_dim)
        for t in range(T):
            x = self._update(x, U[t])
            if t >= washout:
                X[t - washout] = x
        Yt = Y[washout:]
        
        self.Wout = np.linalg.solve(
            X.T.dot(X) + self.ridge * np.eye(self.res_dim),
            X.T.dot(Yt)
        ).T

    def predict_with_drive(self, U, washout=400):
        
        T = U.shape[0]
        x = np.zeros(self.res_dim)
        outs = []
        for t in range(T):
            x = self._update(x, U[t])
            if t >= washout:
                outs.append(self.Wout.dot(x))
        return np.array(outs)


esn_open = ESN(in_dim=3, res_dim=500, out_dim=1,
          spectral_radius=1.2, leaking_rate=0.3,sparsity=0.01,
          ridge_param=1e-7, random_state=42)
washout = 400
esn_open.fit(U_train_s, Y_train_s, washout=washout)



Y_pred_s = esn_open.predict_with_drive(U_test_s, washout=washout)  # (T_test-washout, 1)
Y_pred   = scaler_Y.inverse_transform(Y_pred_s)[:,0]               # (N,)

angle_true = angle_rad[idx_split+washout:]
time_pred = time[idx_split+washout:]


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
sin_pred = np.sin(Y_pred)
cos_pred = np.cos(Y_pred)
sin_true = np.sin(angle_true)
cos_true = np.cos(angle_true)

plt.figure(figsize=(10,3))
plt.plot(time, sin_ang,  label='sin(转角)', color='green', alpha=0.7)
plt.plot(time, cos_ang,  label='cos(转角)', color='purple', alpha=0.7)
plt.xlabel('时间 (s)')
plt.ylabel('数值')
plt.title('原始转角经过 sin / cos 变换后的输入特征')
plt.legend()
plt.tight_layout()
plt.show()


sin_pred = np.sin(Y_pred)
cos_pred = np.cos(Y_pred)

angle_true = angle_rad[idx_split+washout:]  
sin_true = np.sin(angle_true)
cos_true = np.cos(angle_true)

plt.figure(figsize=(10,4))
plt.plot(time_pred, sin_true, label='真实 sin(转角)', color='green', alpha=0.7)
plt.plot(time_pred, sin_pred, '--', label='预测 sin(转角)', color='orange', alpha=0.7)
plt.xlabel('时间 (s)')
plt.ylabel('sin(转角)')
plt.title('sin(转角)：真实 vs. 预测')
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(time_pred, cos_true, label='真实 cos(转角)', color='purple', alpha=0.7)
plt.plot(time_pred, cos_pred, '--', label='预测 cos(转角)', color='red', alpha=0.7)
plt.xlabel('时间 (s)')
plt.ylabel('cos(转角)')
plt.title('cos(转角)：真实 vs. 预测')
plt.legend()
plt.tight_layout()
plt.show()

Y_train_pred_s = esn_open.predict_with_drive(U_train_s, washout=washout)
Y_train_pred = scaler_Y.inverse_transform(Y_train_pred_s)[:,0]
angle_train_true = angle_rad[washout:idx_split]  
time_train_pred = time[washout:idx_split]


plt.figure(figsize=(10,4))
plt.plot(time_train_pred, angle_train_true, label='训练集真实转角', color='blue')
plt.plot(time_train_pred, Y_train_pred, '--', label='训练集预测转角', color='orange')
plt.xlabel('时间 (s)')
plt.ylabel('转角 (rad)')
plt.title('ESN 开放预测（训练集）：真实 vs. 预测')
plt.legend()
plt.tight_layout()
plt.show()


err_train = (Y_train_pred - angle_train_true + np.pi) % (2 * np.pi) - np.pi
err_train = np.abs(err_train)
plt.figure(figsize=(10,3))
plt.plot(time_train_pred, err_train, color='red', label='训练集绝对误差')
plt.xlabel('时间 (s)')
plt.ylabel('绝对误差 (rad)')
plt.title('训练集开放预测误差随时间演化')
plt.legend()
plt.tight_layout()
plt.show()




washout = 800       
idx_split = int(800.0 / dt)  


print(f"U_train_s 长度: {U_train_s.shape[0]}")
print(f"Y_train_s 长度: {Y_train_s.shape[0]}")


Y_train_pred_s = esn_open.predict_with_drive(U_train_s, washout=washout)
print(f"predict_with_drive 输出长度: {Y_train_pred_s.shape[0]}")


angle_train_true = angle_rad[washout:idx_split]
print(f"angle_train_true 长度: {angle_train_true.shape[0]}")


if Y_train_pred_s.shape[0] != angle_train_true.shape[0]:
    print("❗ 长度不匹配，请检查 washout 和 idx_split 是否与 fit() 保持一致。")
else:
    print("✅ 训练集回放输出和真实对齐长度一致。")

Y_train_pred = scaler_Y.inverse_transform(Y_train_pred_s)[:,0]  # (idx_split - washout,)
Y_pred_mod   = np.mod(Y_train_pred, 2*np.pi)
angle_true_mod = np.mod(angle_train_true, 2*np.pi)

err_wrap = (Y_pred_mod - angle_true_mod + np.pi) % (2*np.pi) - np.pi
err_replay = np.abs(err_wrap)

print(f"训练集回放误差（绝对值）均值: {err_replay.mean():.6f} rad")
print(f"训练集回放误差（绝对值）最大: {err_replay.max():.6f} rad")

time_train_pred = time[washout:idx_split]

plt.figure(figsize=(8, 3))
plt.plot(time_train_pred, err_replay, color='red', label='训练集回放误差')
plt.xlabel('时间 (s)（训练集回放）')
plt.ylabel('绝对误差 (rad)')
plt.title('训练集开放预测（回放）误差随时间演化')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

washout = 400       
idx_split = int(800.0 / dt)  


print(f"U_train_s 长度: {U_train_s.shape[0]}")
print(f"Y_train_s 长度: {Y_train_s.shape[0]}")


Y_train_pred_s = esn_open.predict_with_drive(U_train_s, washout=washout)
print(f"predict_with_drive 输出长度: {Y_train_pred_s.shape[0]}")

angle_train_true = angle_rad[washout:idx_split]
print(f"angle_train_true 长度: {angle_train_true.shape[0]}")

if Y_train_pred_s.shape[0] != angle_train_true.shape[0]:
    print("❗ 长度不匹配，请检查 washout 和 idx_split 是否与 fit() 保持一致。")
else:
    print("✅ 训练集回放输出和真实对齐长度一致。")

Y_train_pred = scaler_Y.inverse_transform(Y_train_pred_s)[:,0]  # (idx_split - washout,)
Y_pred_mod   = np.mod(Y_train_pred, 2*np.pi)
angle_true_mod = np.mod(angle_train_true, 2*np.pi)

err_wrap = (Y_pred_mod - angle_true_mod + np.pi) % (2*np.pi) - np.pi
err_replay = np.abs(err_wrap)

print(f"训练集回放误差（绝对值）均值: {err_replay.mean():.6f} rad")
print(f"训练集回放误差（绝对值）最大: {err_replay.max():.6f} rad")

time_train_pred = time[washout:idx_split]

plt.figure(figsize=(8, 3))
plt.plot(time_train_pred, err_replay, color='red', label='训练集回放误差')
plt.xlabel('时间 (s)（训练集回放）')
plt.ylabel('绝对误差 (rad)')
plt.title('训练集开放预测（回放）误差随时间演化')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

plt.figure(figsize=(10,4))
plt.plot(time_pred[::10], sin_true[::10],
         '--', color='black', linewidth=0.5, alpha=0.7,
         label='正弦真实 (subsampled)')
plt.plot(time_pred[::10], sin_pred[::10],
         '--', color='orange', linewidth=0.5, alpha=0.7,
         label='正弦预测 (subsampled)')
plt.axvline(time[idx_split], color='gray', linestyle=':')
plt.xlabel('时间 (s)')
plt.ylim(-1, 1)
plt.ylabel('sin(转角)')
plt.title('ESN 开放预测：sin(转角) 真实 vs. 预测')
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(time_pred[::10], cos_true[::10],
         '--', color='blue', linewidth=0.5, alpha=0.7,
         label='余弦真实 (subsampled)')
plt.plot(time_pred[::10], cos_pred[::10],
         '--', color='green', linewidth=0.5, alpha=0.7,
         label='余弦预测 (subsampled)')
plt.axvline(time[idx_split], color='gray', linestyle=':')
plt.xlabel('时间 (s)')
plt.ylim(-1, 1)
plt.ylabel('cos(转角)')
plt.title('ESN 开放预测：cos(转角) 真实 vs. 预测')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

sin_pred = np.sin(Y_pred)
cos_pred = np.cos(Y_pred)
sin_true = np.sin(angle_true)
cos_true = np.cos(angle_true)

window_sec = 10      
window_steps = int(window_sec / dt)
n_steps = len(time_pred)
n_windows = int(np.ceil(n_steps / window_steps))

out_dir = Path("prediction_plots")
out_dir.mkdir(exist_ok=True)

for i in range(n_windows):
    start = i * window_steps
    end   = min(start + window_steps, n_steps)
    t_win = time_pred[start:end]
    sin_pred_win = sin_pred[start:end]
    sin_true_win = sin_true[start:end]
    cos_pred_win = cos_pred[start:end]
    cos_true_win = cos_true[start:end]

    plt.figure(figsize=(6,3))
    plt.plot(t_win, sin_true_win, '-', color='blue', label='真实 sin(转角)')
    plt.plot(t_win, sin_pred_win, '--', color='orange', label='预测 sin(转角)')
    plt.xlabel('时间 (s)')
    plt.ylabel('sin(转角)')
    plt.ylim(-1, 1)
    plt.title(f'sin(转角)窗口 {i+1}: {t_win[0]:.1f}s–{t_win[-1]:.1f}s')
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()
    # plt.savefig(out_dir / f'sin_window_{i+1}.png')  

    plt.figure(figsize=(6,3))
    plt.plot(t_win, cos_true_win, '-', color='green', label='真实 cos(转角)')
    plt.plot(t_win, cos_pred_win, '--', color='red', label='预测 cos(转角)')
    plt.xlabel('时间 (s)')
    plt.ylabel('cos(转角)')
    plt.ylim(-1, 1)
    plt.title(f'cos(转角)窗口 {i+1}: {t_win[0]:.1f}s–{t_win[-1]:.1f}s')
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()
    # plt.savefig(out_dir / f'cos_window_{i+1}.png')  